In [0]:
pip install nltk

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import re
import string
import nltk
import math
from nltk.corpus import stopwords

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Out[2]: True

In [0]:
plots = sc.textFile("/FileStore/tables/plot_summaries.txt")
movies = sc.textFile("/FileStore/tables/movie_metadata.tsv")
#plots.collect()
#movies.collect()

In [0]:
plots_words = plots.map(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
plots_words = plots_words.map(lambda x: re.split("\t", x.lower()))
plots_words.collect()

Out[4]: [['23890098',
  'shlykov a hardworking taxi driver and lyosha a saxophonist develop a bizarre lovehate relationship and despite their prejudices realize they arent so different after all'],
 ['31186339',
  'the nation of panem consists of a wealthy capitol and twelve poorer districts as punishment for a past rebellion each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual hunger games the tributes must fight to the death in an arena the sole survivor is rewarded with fame and wealth in her first reaping 12yearold primrose everdeen is chosen from district 12 her older sister katniss volunteers to take her place peeta mellark a bakers son who once gave katniss bread when she was starving is the other district 12 tribute katniss and peeta are taken to the capitol accompanied by their frequently drunk mentor past victor haymitch abernathy he warns them about the career tributes who train intensively at special academies and almo

In [0]:
plots_separated = plots_words.flatMap(lambda x : [((x[0], i.lower()), 1) for i in re.split(" ", x[1]) if len(i)>1])
plots_separated.take(10)

Out[5]: [(('23890098', 'shlykov'), 1),
 (('23890098', 'hardworking'), 1),
 (('23890098', 'taxi'), 1),
 (('23890098', 'driver'), 1),
 (('23890098', 'and'), 1),
 (('23890098', 'lyosha'), 1),
 (('23890098', 'saxophonist'), 1),
 (('23890098', 'develop'), 1),
 (('23890098', 'bizarre'), 1),
 (('23890098', 'lovehate'), 1)]

In [0]:
plots_stopwords = plots_separated.filter(lambda x: x[0][1] not in nltk.corpus.stopwords.words("english"))
plots_stopwords.take(10)

Out[6]: [(('23890098', 'shlykov'), 1),
 (('23890098', 'hardworking'), 1),
 (('23890098', 'taxi'), 1),
 (('23890098', 'driver'), 1),
 (('23890098', 'lyosha'), 1),
 (('23890098', 'saxophonist'), 1),
 (('23890098', 'develop'), 1),
 (('23890098', 'bizarre'), 1),
 (('23890098', 'lovehate'), 1),
 (('23890098', 'relationship'), 1)]

In [0]:
term_frequency = plots_stopwords.reduceByKey(lambda x,y: x + y)
term_frequency.take(10)

Out[7]: [(('2231378', 'license'), 3),
 (('24225279', 'recognizes'), 1),
 (('1335380', 'someday'), 2),
 (('5414895', 'start'), 1),
 (('17124781', 'raised'), 1),
 (('4466226', 'fyodor'), 7),
 (('17711304', 'divert'), 1),
 (('15161056', 'breaking'), 1),
 (('18369853', 'away'), 3),
 (('529276', 'banana'), 1)]

In [0]:
term_count = term_frequency.map(lambda x: (x[0][1],1))
document_frequency = term_count.reduceByKey(lambda x,y: x+y)
document_frequency.take(10)

Out[8]: [('plot', 3115),
 ('tells', 9531),
 ('ever', 1920),
 ('stoic', 23),
 ('sing', 635),
 ('entire', 1998),
 ('work', 5634),
 ('subdued', 105),
 ('house', 7517),
 ('set', 5124)]

In [0]:
count_documents = plots.map(lambda x:(1,1))
total_documents = count_documents.reduceByKey(lambda x,y: x+y).collect()
total_documents = total_documents[0][1]

In [0]:
log_N_by_document_frequency = document_frequency.map(lambda x: (x[0],math.log(total_documents/x[1])))
log_N_by_document_frequency.collect()

Out[10]: [('plot', 2.608699767725542),
 ('tells', 1.4903792760094696),
 ('ever', 3.0926037339252685),
 ('stoic', 7.517189983017946),
 ('sing', 4.199059200054404),
 ('entire', 3.0527822397385966),
 ('work', 2.0161092505107776),
 ('subdued', 5.998723848789572),
 ('house', 1.7277617977696267),
 ('set', 2.110993535930471),
 ('janies', 9.266389837827205),
 ('wedding', 3.1036014881348097),
 ('suburban', 5.155515973653894),
 ('using', 2.480519746835192),
 ('based', 3.300884329889319),
 ('fires', 3.9004138228053535),
 ('explosion', 4.097327307136431),
 ('hanks', 7.156176637480615),
 ('smitty', 8.08773484148556),
 ('husbandry', 9.554071910278985),
 ('although', 2.601980817476797),
 ('human', 3.1916186845928127),
 ('end', 1.8548335500160416),
 ('stated', 5.635404362132172),
 ('quiet', 4.186539474709476),
 ('rush', 4.333716085200661),
 ('mysterious', 3.055789760802552),
 ('would', 2.116080614011035),
 ('propelled', 6.989122552817449),
 ('figure', 3.66796787882883),
 ('sugar', 5.899094007840731),


In [0]:
term_frequency_for_join = term_frequency.map(lambda x: (x[0][1],(x[0][0],x[1])))

In [0]:
tf_idf = term_frequency_for_join.join(log_N_by_document_frequency)

In [0]:
tf_idf_values = tf_idf.map(lambda x: ((x[1][0][0],x[0]),x[1][0][1]*x[1][1])).collect()
tf_idf_data = sc.parallelize(tf_idf_values)

In [0]:
movies_metadata = movies.map(lambda x: (x.split('\t')[0],x.split('\t')[2]))

In [0]:
query_word = "Mary"
input_filter = tf_idf_data.filter(lambda x: x[0][1].lower()==query_word.lower())
input_filter.map(lambda x: (x[0][0],x[1])).join(movies_metadata).sortBy(lambda x: -x[1][0]).map(lambda x: x[1][1]).take(10)

Out[15]: ['Prom Night II',
 'La Mary',
 'Saved!',
 'The Wedding Planner',
 'Psycho II',
 "I'll Be Seeing You",
 'Prime Cut',
 'The Mary Kay Letourneau Story: All-American Girl',
 'Mary Poppins',
 'The Women']

In [0]:
query_sentence = "Wall Street"
query_sentence = query_sentence.translate(str.maketrans('', '', string.punctuation))
query_sentence = [word for word in query_sentence.lower().split(" ") if not word in nltk.corpus.stopwords.words("english")]
query_sentence_rdd = sc.parallelize(query_sentence)
query_sentence_rdd = query_sentence_rdd.filter(lambda x: x!='')
words_to_search = query_sentence_rdd.collect()

In [0]:
query_idf = query_sentence_rdd.map(lambda x: (x,1)).reduceByKey(lambda x,y : x+y).join(log_N_by_document_frequency)
query_idf.collect()

Out[17]: [('street', (1, 3.105710081430569)), ('wall', (1, 3.553482455394003))]

In [0]:
query_tf = query_sentence_rdd.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
query_tf.collect()

Out[18]: [('wall', 1), ('street', 1)]

In [0]:
query_tf_idf = query_tf.join(query_idf).map(lambda x: (x[0],x[1][0]*x[1][1][1])).reduceByKey(lambda x,y: x+y)
query_tf_idf.collect()

Out[19]: [('wall', 3.553482455394003), ('street', 3.105710081430569)]

In [0]:
square_document_tfidf = tf_idf_data.map(lambda x:(x[0][0],x[1]*x[1])).reduceByKey(lambda x,y: x + y)
square_query_tfidf = query_tf_idf.map(lambda x:(x[1]*x[1])).reduce(lambda x,y: x + y) 
data_cos_sim = tf_idf_data.filter(lambda x: x[0][1].lower() in words_to_search)
data_cos_sim.collect()

Out[20]: [(('18619636', 'wall'), 3.553482455394003),
 (('15161056', 'wall'), 3.553482455394003),
 (('36200448', 'wall'), 3.553482455394003),
 (('33061601', 'wall'), 3.553482455394003),
 (('15242784', 'wall'), 7.106964910788006),
 (('34010844', 'wall'), 10.660447366182009),
 (('10131328', 'wall'), 7.106964910788006),
 (('19062669', 'wall'), 3.553482455394003),
 (('2521806', 'wall'), 3.553482455394003),
 (('548259', 'wall'), 3.553482455394003),
 (('1251285', 'wall'), 3.553482455394003),
 (('15731217', 'wall'), 3.553482455394003),
 (('15721024', 'wall'), 3.553482455394003),
 (('6562375', 'wall'), 7.106964910788006),
 (('957991', 'wall'), 3.553482455394003),
 (('7498130', 'wall'), 7.106964910788006),
 (('2063141', 'wall'), 3.553482455394003),
 (('9362805', 'wall'), 3.553482455394003),
 (('3286162', 'wall'), 7.106964910788006),
 (('5995578', 'wall'), 3.553482455394003),
 (('6410861', 'wall'), 3.553482455394003),
 (('12139040', 'wall'), 3.553482455394003),
 (('25431279', 'wall'), 3.553482455

In [0]:
data_cos_sim = data_cos_sim.map(lambda x: (x[0][1],(x[0][0],x[1])))
result = data_cos_sim.join(query_tf_idf)
result.collect()

Out[21]: [('wall', (('18619636', 3.553482455394003), 3.553482455394003)),
 ('wall', (('15161056', 3.553482455394003), 3.553482455394003)),
 ('wall', (('36200448', 3.553482455394003), 3.553482455394003)),
 ('wall', (('33061601', 3.553482455394003), 3.553482455394003)),
 ('wall', (('15242784', 7.106964910788006), 3.553482455394003)),
 ('wall', (('34010844', 10.660447366182009), 3.553482455394003)),
 ('wall', (('10131328', 7.106964910788006), 3.553482455394003)),
 ('wall', (('19062669', 3.553482455394003), 3.553482455394003)),
 ('wall', (('2521806', 3.553482455394003), 3.553482455394003)),
 ('wall', (('548259', 3.553482455394003), 3.553482455394003)),
 ('wall', (('1251285', 3.553482455394003), 3.553482455394003)),
 ('wall', (('15731217', 3.553482455394003), 3.553482455394003)),
 ('wall', (('15721024', 3.553482455394003), 3.553482455394003)),
 ('wall', (('6562375', 7.106964910788006), 3.553482455394003)),
 ('wall', (('957991', 3.553482455394003), 3.553482455394003)),
 ('wall', (('7498130',

In [0]:
result = result.map(lambda x: (x[1][0][0], (x[1][0][1], x[1][1])))
result = result.join(square_document_tfidf)
result.collect()

Out[22]: [('18790154', ((7.106964910788006, 3.553482455394003), 61599.45031162146)),
 ('10141485', ((7.106964910788006, 3.553482455394003), 19423.47589914646)),
 ('30081969', ((3.553482455394003, 3.553482455394003), 49783.11593634267)),
 ('30081969', ((3.105710081430569, 3.105710081430569), 49783.11593634267)),
 ('212265', ((3.553482455394003, 3.553482455394003), 54439.57938844136)),
 ('161334', ((10.660447366182009, 3.553482455394003), 22498.185351586246)),
 ('161334', ((12.422840325722277, 3.105710081430569), 22498.185351586246)),
 ('8896262', ((3.553482455394003, 3.553482455394003), 157415.51322409962)),
 ('13610020', ((3.553482455394003, 3.553482455394003), 94689.55925119903)),
 ('22572873', ((3.553482455394003, 3.553482455394003), 987.0631347137994)),
 ('23199746', ((17.767412276970017, 3.553482455394003), 43682.21463768563)),
 ('6277939', ((10.660447366182009, 3.553482455394003), 294413.957600886)),
 ('4181412', ((7.106964910788006, 3.553482455394003), 50610.800535683826)),
 ('28

In [0]:
temp_result = result.map(lambda x: (x[0], (x[1][0][0]*x[1][0][1])/(math.sqrt(x[1][1])*math.sqrt(square_query_tfidf))))
cos_sim = temp_result.reduceByKey(lambda x,y: x+y)
cos_sim.collect()

Out[23]: [('18790154', 0.021560744897945783),
 ('10141485', 0.03839624056095139),
 ('30081969', 0.021151692756163),
 ('212265', 0.011467395819195283),
 ('161334', 0.10801746841549768),
 ('8896262', 0.006743702747843927),
 ('13610020', 0.008695031031653356),
 ('22572873', 0.08516275803720809),
 ('23199746', 0.06400882631529686),
 ('6277939', 0.014793271943742681),
 ('4181412', 0.023786501775977),
 ('2886917', 0.03169008222566232),
 ('2259688', 0.008939632860854713),
 ('5961506', 0.030756529046315186),
 ('9395068', 0.01775034160837527),
 ('14157327', 0.034221921633543546),
 ('14737096', 0.05320520292051414),
 ('438548', 0.022726511222685614),
 ('28001277', 0.023509992186785504),
 ('27485768', 0.011777190016536824),
 ('142274', 0.01558210395927059),
 ('2220694', 0.03481054234561423),
 ('32584198', 0.0624150590242518),
 ('4213160', 0.019591164338069938),
 ('15107077', 0.07449197906757794),
 ('27609842', 0.007904560907360194),
 ('9186440', 0.020086323842176824),
 ('5894429', 0.0296989784554

In [0]:
top_10_docs = cos_sim.sortBy(lambda x: -x[1])
top_10_docs.join(movies_metadata).sortBy(lambda x: -x[1][0]).map(lambda x:(x[1][1],x[1][0])).take(10)

Out[25]: [('Knife Edge', 0.3526018176491698),
 ('The Wall', 0.3253290685097485),
 ('Katha Parayum Theruvoram', 0.25374722877696193),
 ('Think Before You Pink', 0.24629016509765023),
 ('Changing the Game', 0.2287768640446502),
 ('The Reluctant Fundamentalist', 0.2286939480905129),
 ('Honor Among Lovers', 0.21812924768819725),
 ('Reaching for the Moon', 0.196327293199937),
 ('The Transporter', 0.19031283146367872),
 ('Wolves of Wall Street', 0.18363202045567606)]